In [8]:
import os
import json
import shutil
from tqdm import tqdm
from PIL import Image

# ===== 1. المسارات =====
img_dir = "/kaggle/input/deepfashion2-original-with-dataframes/DeepFashion2/deepfashion2_original_images/train/image"
ann_dir = "/kaggle/input/deepfashion2-original-with-dataframes/DeepFashion2/deepfashion2_original_images/train/annos"

# فولدر الإخراج
output_img = "/kaggle/working/dataset/images/train"
output_lbl = "/kaggle/working/dataset/labels/train"

os.makedirs(output_img, exist_ok=True)
os.makedirs(output_lbl, exist_ok=True)

# ===== 2. إعداد الكلاسات =====
categories = [
  "short_sleeve_top", "long_sleeve_top", "short_sleeve_outwear", 
  "long_sleeve_outwear", "vest", "sling", "shorts", "trousers",
  "skirt", "short_sleeve_dress", "long_sleeve_dress", "vest_dress",
  "sling_dress"
]

# ===== 3. التحويل =====
all_files = [f for f in os.listdir(ann_dir) if f.endswith(".json")]

def process_file(ann_file):
    ann_path = os.path.join(ann_dir, ann_file)
    with open(ann_path, "r") as f:
        ann = json.load(f)

    img_file = ann_file.replace(".json", ".jpg")
    img_path = os.path.join(img_dir, img_file)

    if not os.path.exists(img_path):
        return
    
    try:
        img = Image.open(img_path)
        img_w, img_h = img.size
    except:
        return

    lines = []
    for key in ann:
        if key.startswith("item"):  
            item = ann[key]
            cat_id = int(item["category_id"]) - 1  # YOLO يبدأ من 0
            
            # bbox: [x1, y1, x2, y2]
            x1, y1, x2, y2 = item["bounding_box"]
            w = x2 - x1
            h = y2 - y1
            x_c = x1 + w/2
            y_c = y1 + h/2

            # نسبية
            x_c /= img_w
            y_c /= img_h
            w /= img_w
            h /= img_h

            lines.append(f"{cat_id} {x_c:.6f} {y_c:.6f} {w:.6f} {h:.6f}")

    if not lines:
        return

    # انسخ الصورة
    shutil.copy(img_path, os.path.join(output_img, img_file))
    # اكتب الـ label
    with open(os.path.join(output_lbl, ann_file.replace(".json", ".txt")), "w") as f:
        f.write("\n".join(lines))

# Loop
for f in tqdm(all_files):
    process_file(f)

print("✅ التحويل انتهى، كل الصور والـ labels في /kaggle/working/dataset")


100%|██████████| 191961/191961 [1:08:32<00:00, 46.68it/s]

✅ التحويل انتهى، كل الصور والـ labels في /kaggle/working/dataset


In [9]:
import os
import json
import shutil
from tqdm import tqdm
from PIL import Image

# ===== 1. المسارات =====
img_dir = "/kaggle/input/deepfashion2-original-with-dataframes/DeepFashion2/deepfashion2_original_images/validation/image"
ann_dir = "/kaggle/input/deepfashion2-original-with-dataframes/DeepFashion2/deepfashion2_original_images/validation/annos"

# فولدر الإخراج
output_img = "/kaggle/working/dataset/images/val"
output_lbl = "/kaggle/working/dataset/labels/val"

os.makedirs(output_img, exist_ok=True)
os.makedirs(output_lbl, exist_ok=True)

# ===== 2. إعداد الكلاسات =====
categories = [
  "short_sleeve_top", "long_sleeve_top", "short_sleeve_outwear", 
  "long_sleeve_outwear", "vest", "sling", "shorts", "trousers",
  "skirt", "short_sleeve_dress", "long_sleeve_dress", "vest_dress",
  "sling_dress"
]

# ===== 3. التحويل =====
all_files = [f for f in os.listdir(ann_dir) if f.endswith(".json")]

def process_file(ann_file):
    ann_path = os.path.join(ann_dir, ann_file)
    with open(ann_path, "r") as f:
        ann = json.load(f)

    img_file = ann_file.replace(".json", ".jpg")
    img_path = os.path.join(img_dir, img_file)

    if not os.path.exists(img_path):
        return
    
    try:
        img = Image.open(img_path)
        img_w, img_h = img.size
    except:
        return

    lines = []
    for key in ann:
        if key.startswith("item"):  
            item = ann[key]
            cat_id = int(item["category_id"]) - 1  # YOLO يبدأ من 0
            
            # bbox: [x1, y1, x2, y2]
            x1, y1, x2, y2 = item["bounding_box"]
            w = x2 - x1
            h = y2 - y1
            x_c = x1 + w/2
            y_c = y1 + h/2

            # نسبية
            x_c /= img_w
            y_c /= img_h
            w /= img_w
            h /= img_h

            lines.append(f"{cat_id} {x_c:.6f} {y_c:.6f} {w:.6f} {h:.6f}")

    if not lines:
        return

    
    shutil.copy(img_path, os.path.join(output_img, img_file))
   
    with open(os.path.join(output_lbl, ann_file.replace(".json", ".txt")), "w") as f:
        f.write("\n".join(lines))


for f in tqdm(all_files):
    process_file(f)

print("✅ التحويل انتهى، كل الصور والـ labels في /kaggle/working/dataset")


100%|██████████| 32153/32153 [11:01<00:00, 48.59it/s]

✅ التحويل انتهى، كل الصور والـ labels في /kaggle/working/dataset


In [10]:
yaml_content = f"""
train: /kaggle/working/dataset/images/train
val: /kaggle/working/dataset/images/val
nc: {len(categories)}
names: {categories}
"""

with open("/kaggle/working/dataset.yaml", "w") as f:
    f.write(yaml_content)

print("✅ dataset.yaml جاهز")


✅ dataset.yaml جاهز


In [11]:
import os
import shutil

# تأكدي إن مجلد outputs موجود
os.makedirs("/kaggle/outputs", exist_ok=True)

# انسخي الملف
shutil.copy("/kaggle/working/dataset.yaml", "/kaggle/outputs/dataset.yaml")

print("✅ اتنسخ الملف في /kaggle/outputs/dataset.yaml")



✅ اتنسخ الملف في /kaggle/outputs/dataset.yaml


In [12]:
import os

print(os.path.exists("/kaggle/outputs/dataset.yaml"))
!ls -lh /kaggle/outputs

True
total 4.0K
-rw-r--r-- 1 root root 306 Sep  5 11:17 dataset.yaml


In [13]:
pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.1 MB/s eta 0:00:00:00:0100:01
  Attempting uninst

In [14]:
from ultralytics import YOLO
model = YOLO("yolov8m.pt")  

model.train(
    data="/kaggle/working/dataset.yaml",  
    epochs=3,       
    batch=16,         
    imgsz=640,       
    workers=2         
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.193 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=3, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, 

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all      32153      52490      0.763      0.739      0.797      0.698
      short_sleeve_top      12371      12556      0.865      0.919      0.955      0.865
       long_sleeve_top       5916       5966       0.78      0.837      0.883      0.778
  short_sleeve_outwear        142        142      0.586      0.269      0.385      0.347
   long_sleeve_outwear       1999       2011      0.734      0.871       0.85      0.766
                  vest       2098       2113      0.698      0.832      0.848      0.722
                 sling        317        322      0.763      0.449      0.588      0.472
                shorts       4133       4167      0.866      0.857      0.926      0.767
              trousers       9494       9586       0.89      0.937      0.966      0.805
                 skirt       6499       6522      0.812      0.862      0.905      0.795
    short_sleeve_dress       3076       3127      0.759      0.761      0.828      0.771
     long_sleeve_dres

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7bec8d25e5d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.04504

In [ ]:
import shutil

# copy best weights to a custom location
shutil.copy("runs/detect/train/weights/best.pt", "model.pt")

'/kaggle/working/my_model.pt'